# Collaborative Filtering with Neural Nets

In [7]:
# for data manipulation
import numpy as np
import pandas as pd
import os
import pickle

# use surprise for collaborative filtering
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable


# plot
import matplotlib.pyplot as plt

## Read in data

In [8]:
game_data_path = "data/neural_net_data/"
files = os.listdir(game_data_path)
with open(game_data_path + files[0], 'rb') as f:
    X, y = pickle.load(f)

In [9]:
X.shape

(1267, 3, 504)

In [10]:
y.shape

(1267,)

## Clean data

In [11]:
X = X[y > 0]
y = y[y > 0]

p = np.random.permutation(len(X))
X = X[p]
y = y[p]

val = 0.2
val = round(len(X) * val)
val_X = X[:val]
val_y = y[:val]
X = X[val:]
y = y[val:]

In [12]:
#### Specify the model architecture
class LSTMModel(nn.Module):

    def __init__(self, input_dim, hidden_dim, target_size, num_layers, batch_size, time_steps):
        super(LSTMModel, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.time_steps = time_steps
        
        # Initialize LSTM unit
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=False)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2out = nn.Linear(hidden_dim, target_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size , hidden_dim)
        return (torch.zeros(self.num_layers, self.time_steps, self.hidden_dim),
                torch.zeros(self.num_layers, self.time_steps, self.hidden_dim))
        
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))

    def forward(self, input_seq):
        lstm_out, self.hidden = self.lstm(input_seq, self.hidden)
        
        pred = self.hidden2out(lstm_out)
        
        return pred

In [13]:
## Define the model
model = LSTMModel(input_dim = 504,
                     hidden_dim = 20,
                     target_size = 1,
                     num_layers = 1,
                     batch_size = 10, 
                     time_steps = 3)
                     
loss_function = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [14]:
losses = []
val_losses = []

for epoch in range(500):   # again, normally you would NOT do 300 epochs, it is toy data
    train_loss = 0
    for i in range(0, len(X), model.batch_size):
        if i + model.batch_size >= len(X) : continue
        
        #Pytorch accumulates gradients. We need to clear them out before each instance
        model.zero_grad()

        # Also, we need to clear out the hidden state of the LSTM, detaching it from its history on the last instance.
        model.hidden = model.init_hidden()

        # Step 2. Get our inputs ready for the network.
        batch_input = X[i : i + model.batch_size] #.reshape((X.shape[1], model.batch_size, X.shape[2]))
        batch = Variable(torch.from_numpy(batch_input)).type(torch.FloatTensor)
                                                    
        targets = Variable(torch.from_numpy(y[i : i + model.batch_size])).type(torch.FloatTensor)

        # Step 3. Run our forward pass.
        scores = model(batch)
        scores = scores[:, -1].reshape((model.batch_size)) # we only care about the last output

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(scores, targets)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.detach().numpy()
        
    ## validation loss
    if (epoch + 1) % 10 == 0:
        print("----------")
        print("Losses after {} iterations:".format(epoch))
        print("Train: {}".format(loss.detach().numpy()))
#        with torch.no_grad():
#             batch_input = val_X
#             batch = Variable(torch.from_numpy(batch_input)).type(torch.FloatTensor)
#             targets = Variable(torch.from_numpy(val_y)).type(torch.FloatTensor)
#             scores = model(batch)
#             scores = scores[:, -1].reshape((len(val_y))) # we only care about the last output
#             val_loss = loss_function(scores, targets)
#             print("Val: {}".format(val_loss))
#             val_losses.append(val_loss)
#             losses.append(train_loss/len(X))

----------
Losses after 9 iterations:
Train: 262.7909240722656
----------
Losses after 19 iterations:
Train: 266.32965087890625
----------
Losses after 29 iterations:
Train: 267.8335876464844
----------
Losses after 39 iterations:
Train: 268.3408508300781
----------
Losses after 49 iterations:
Train: 268.5025939941406
----------
Losses after 59 iterations:
Train: 268.5531921386719
----------
Losses after 69 iterations:
Train: 268.56915283203125
----------
Losses after 79 iterations:
Train: 268.5740661621094
----------
Losses after 89 iterations:
Train: 268.575439453125
----------
Losses after 99 iterations:
Train: 268.5760192871094
----------
Losses after 109 iterations:
Train: 268.57611083984375
----------
Losses after 119 iterations:
Train: 268.576171875
----------
Losses after 129 iterations:
Train: 268.576171875
----------
Losses after 139 iterations:
Train: 268.57623291015625
----------
Losses after 149 iterations:
Train: 268.5761413574219
----------
Losses after 159 iterations:
T

KeyboardInterrupt: 

In [ ]:
# See what the scores are after training
with torch.no_grad():
    batch_input = val_X
    batch = Variable(torch.from_numpy(batch_input)).type(torch.FloatTensor)
    targets = Variable(torch.from_numpy(val_y)).type(torch.FloatTensor)
    scores = model(batch)
    scores = scores[:, -1].reshape((len(val_y))) # we only care about the last output
    val_loss = loss_function(scores, targets)
    print(val_loss)